<a href="https://colab.research.google.com/github/amunoz88/Applied_DS_2025/blob/main/Privacy_explorations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import dataclasses
import numpy as np

In [ ]:
data =pd.read_csv('/content/adult.csv')
data['rounded_age'] = data['age']//10 * 10

In [ ]:
data.occupation.unique()

array(['Machine-op-inspct', 'Farming-fishing', 'Protective-serv', '?',
       'Other-service', 'Prof-specialty', 'Craft-repair', 'Adm-clerical',
       'Exec-managerial', 'Tech-support', 'Sales', 'Priv-house-serv',
       'Transport-moving', 'Handlers-cleaners', 'Armed-Forces'],
      dtype=object)

In [ ]:
data_small = data[['rounded_age', 'gender', 'race', 'relationship', 'workclass', 'income']]

In [ ]:
data_small.loc[:, 'rounded_age'] = data_small.rounded_age.astype(str)

<ipython-input-5-5e71eb7d748a>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['20' '30' '20' ... '50' '20' '50']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_small.loc[:, 'rounded_age'] = data_small.rounded_age.astype(str)


In [ ]:
from typing import Union
@dataclasses.dataclass
class Node:
  column: str
  value: str | int
  data : pd.DataFrame | None = None
  children: list['Node'] | None = None
  is_k_anon: bool | None = None
  parent: Union['Node', None] = None
  def add_child(self, child):
    if self.children == None:
      self.children = []
    self.children.append(child)

In [ ]:
import numpy as np
def SplitData(column: str, parent:Node, k: int):
  for v in parent.data[column].unique():
    new_node = Node(column, v, parent=parent, data=parent.data[parent.data[column] == v])
    new_node.is_k_anon = len(new_node.data) >= k
    parent.add_child(new_node)

def SanitizeData(data, columns):
  out = data.copy()
  out.loc[:, columns] = "*"
  return out

def is_split_good(parent: Node):
  return np.sum([n.is_k_anon for n in parent.children]) == len(parent.children)

def AnonimizeTree(root: Node, columns: list[str], k: int):
  # Return condition - We iterated over all columns
  if len(columns) == 0:
    return [root.data]
  column = columns.pop()
  # Return condition - We arrived at a column that when split, fails to create
  # anonymous data.
  SplitData(column, root, k)
  if not is_split_good(root):
    return [SanitizeData(root.data, columns + [column])]

  # If split generates k anonymous data, then do a recursion on all children
  out = []
  for n in root.children:
    out += AnonimizeTree(n, columns.copy(), k)
  return out


def AnonymizeDataset(data, columns, k):
  root = Node(None, None, data=data)
  anonymized_data =  AnonimizeTree(root, columns, k)
  anonymized_data = pd.concat(anonymized_data)
  num_entries_removed = np.sum(np.sum(anonymized_data == "*"))
  return anonymized_data, num_entries_removed

In [ ]:
AnonymizeDataset(data_small, list(data_small.columns), 10)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


(      rounded_age gender   race relationship workclass income
 0               *   Male  Black    Own-child   Private  <=50K
 51              *   Male  Black    Own-child   Private  <=50K
 102             *   Male  Black    Own-child   Private  <=50K
 108             *   Male  Black    Own-child   Private  <=50K
 286             *   Male  Black    Own-child   Private  <=50K
 ...           ...    ...    ...          ...       ...    ...
 48820           *      *      *            *         *   >50K
 48826           *      *      *            *         *   >50K
 48835           *      *      *            *         *   >50K
 48838           *      *      *            *         *   >50K
 48841           *      *      *            *         *   >50K
 
 [48842 rows x 6 columns],
 np.int64(124933))

In [ ]:
import itertools

best_p = None
num_stars = 2**32
best_data = None
features = set(data_small.columns) - set(['income'])
for p in itertools.permutations(features):
  data, total_stars = AnonymizeDataset(data_small, list(p), 10)
  if total_stars < num_stars:
    num_stars = total_stars
    best_p = p
    best_data = data


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs

In [ ]:
print(best_p)

('workclass', 'gender', 'relationship', 'race', 'rounded_age')


In [ ]:
#@title Differential privacy - Randomized response
def GetAnonymousColumn(data, column):
  temp = data[column].loc[np.logical_not(data[column].isna())]
  n = len(temp)
  values = temp.unique()
  assert len(values) == 2
  coinflip1 = np.random.binomial(1, 0.5, n)
  coinflip2 = np.random.binomial(1, 0.5, n)
  true_values_bin = np.zeros(n)
  true_values_bin[temp == values[1]] = 1

  mask = true_values_bin * coinflip1 + (1 - coinflip1) * coinflip2

  out = temp.copy()
  out.loc[mask == 0] = values[0]
  out.loc[mask == 1] = values[1]

  return out
def correction(counts):
  n = np.sum(counts)
  return 2 * (counts -  n/4)

In [ ]:
print(data_small.income.value_counts())
anonymous = GetAnonymousColumn(data_small, 'income')
print(anonymous.value_counts())
print(correction(anonymous.value_counts()))

income
<=50K    37155
>50K     11687
Name: count, dtype: int64
income
<=50K    30708
>50K     18134
Name: count, dtype: int64
income
<=50K    36995.0
>50K     11847.0
Name: count, dtype: float64


In [ ]:
#@title Differential privacy - Laplace mechanism

def GetAnonymousSum(data, column, epsilon):
  true_counts = data[column].value_counts()
  sensitivity = 1
  print(true_counts)
  noise = np.random.laplace(sensitivity/epsilon, len(true_counts))
  true_counts += noise
  return true_counts

In [ ]:
GetAnonymousSum(data_small, 'income', 1.0)

income
<=50K    37155
>50K     11687
Name: count, dtype: int64


,count
income,
<=50K,37158.031843
>50K,11690.031843
